# Crowler

In [1]:
!pip install google-play-scraper

     |████████████████████████████████| 52 kB 579 kB/s 
  Created wheel for google-play-scraper: filename=google_play_scraper-1.0.1-py3-none-any.whl size=24193 sha256=1452814228b3b212da15ad8f6ea4f0fe8eaf4e9662e7c6827090858e2a50c697
  Stored in directory: /root/.cache/pip/wheels/05/8f/af/b45bcc354aac4f16e943658c14517ca439f5b18b766361a260
Successfully built google-play-scraper


In [2]:
app_name= 'br.com.zeroum.turmadamonica' #id do app na play store

In [3]:
from google_play_scraper import Sort, reviews_all
import pandas as pd

result = reviews_all(
  app_name,
  sleep_milliseconds=0,
  lang='en', 
  sort=Sort.NEWEST,
)

df_app_review=pd.DataFrame(result)
df_app_review

,reviewId,userName,userImage,content,score,thumbsUpCount,reviewCreatedVersion,at,replyContent,repliedAt
0,gp:AOqpTOFyRa5DW9IITumBRG7kPCi8q-m7RtDx9rWwrSx...,Nalbert To play!!!,https://play-lh.googleusercontent.com/a-/AOh14...,This game is very good but not to put the epis...,5,2,2.5.3,2021-05-23 17:44:33,None,None
1,gp:AOqpTOH0D9t-byAia9kuLiG4IPmZhiojVICe3cGvM6K...,Josh verduzco,https://play-lh.googleusercontent.com/a/AATXAJ...,☺☺☺☺☺☺☺☺qoop qop. qop?,5,2,None,2021-03-30 23:11:09,None,None
2,gp:AOqpTOFv-ilsMiUsWTPvNeiolpWhA-rYUg_fnIJjY4N...,Silva Kabamba,https://play-lh.googleusercontent.com/a-/AOh14...,A Mei turma da Mônica 😊😊😊😊😊😘,5,10,2.5,2020-05-23 23:51:39,None,None
3,gp:AOqpTOGa1jHHreayvDX46GS-PmCy7CeZz3kCf6F5s62...,anca lupu,https://play-lh.googleusercontent.com/a/AATXAJ...,Monica toy ❤️,5,10,None,2020-02-21 16:08:03,None,None
4,gp:AOqpTOFk3e4uq5G2HnEkmJrY8MBcp9SaB_85r3o1NWz...,Erick Feliciano,https://play-lh.googleusercontent.com/a/AATXAJ...,funny cartoon show.,5,19,1.9.1,2019-10-31 00:14:33,None,None
5,gp:AOqpTOGBcTYuvyQFLEzB2PPCqga23WXgacSa5OAbumT...,Tony Horton,https://play-lh.googleusercontent.com/a-/AOh14...,😍😍😍😍😍😍😍😍😍😍😍😍😍😍😍🍦😍😍😍😍😍😍😍😍😍😍😍😍😍😑😍😍😍😍,5,11,None,2019-06-01 22:34:34,None,None
6,gp:AOqpTOGRZBbgvf6jzgHc0BlzAt0mfhiAQzwPeJjoWQE...,Luciana Pontes,https://play-lh.googleusercontent.com/a-/AOh14...,Bomdiapraquemgostadese,5,15,None,2018-08-25 19:45:45,None,None


# Load Model

In [4]:
!wget https://raw.githubusercontent.com/adailtonaraujo/app_review_analysis/master/Utility/Model/BBR_MDistilBERT.pkl

--2021-08-05 22:10:58--  https://raw.githubusercontent.com/adailtonaraujo/app_review_analysis/master/Utility/Model/BBR_MDistilBERT.pkl
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2110300 (2.0M) [application/octet-stream]
Saving to: ‘BBR_MDistilBERT.pkl’

BBR_MDistilBERT.pkl 100%[===================>]   2.01M  --.-KB/s    in 0.03s   

2021-08-05 22:10:59 (58.9 MB/s) - ‘BBR_MDistilBERT.pkl’ saved [2110300/2110300]



In [5]:
pkl_filename = "BBR_MDistilBERT.pkl"

In [6]:
import pickle
with open(pkl_filename, 'rb') as file:
  rgs = pickle.load(file)

# Represent User Text (USE GPU)

In [7]:
!pip install -U sentence-transformers

     |████████████████████████████████| 85 kB 4.0 MB/s 
     |████████████████████████████████| 2.6 MB 36.3 MB/s 
     |████████████████████████████████| 1.2 MB 30.7 MB/s 
     |████████████████████████████████| 43 kB 2.1 MB/s 
     |████████████████████████████████| 895 kB 46.3 MB/s 
     |████████████████████████████████| 3.3 MB 51.4 MB/s 
     |████████████████████████████████| 636 kB 66.2 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.0.0-py3-none-any.whl size=126709 sha256=4e2be428e1211ab42a080f2d1d568bfccb46eb1fab5e05ca3711fa022a98bbbe
  Stored in directory: /root/.cache/pip/wheels/d1/c1/0f/faafd427f705c4b012274ba60d9a91d75830306811e1355293
Successfully built sentence-transformers
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [8]:
from sentence_transformers import SentenceTransformer
import numpy as np

In [9]:
import pandas as pd
def WordEmbeddings(texts):

  model = SentenceTransformer('distiluse-base-multilingual-cased')

  if type(texts) == pd.core.series.Series:
    sentences = texts.replace(['\t','\n','\r'], [' ',' ',' '], regex=True)
  else:
    sentences = texts

  sentence_embeddings = model.encode(list(sentences))
  
  return sentence_embeddings

The variable *txts* can be a dataframe column or a list of strings

# DEMO


In [10]:
data = WordEmbeddings(df_app_review.content)

predicted_utilites = rgs.predict(data)
predicted_utilites

array([-0.00169089,  0.00181116,  0.00231536,  0.00117951,  0.00134181,
        0.00243526,  0.00219668])

# Utility Metrics

In [11]:
from sklearn.metrics import mean_absolute_error as mae
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import r2_score as r2

In [12]:
y_test_utility = [0, 0, 0, 0, 0, 0, 0]

## Mean Absolute Error

In [13]:
mae(y_test_utility,predicted_utilites)

0.001852951784555009

## Mean Squared Error

In [14]:
mse(y_test_utility,predicted_utilites)

3.6354081079090644e-06

## r2

In [15]:
r2(y_test_utility,predicted_utilites)

0.0